In [ ]:
%load_ext watermark


In [ ]:
from colorclade import draw_colorclade_tree
from matplotlib import pyplot as plt
import numpy as np
from tqdm import tqdm

from pylib._build_hstrat_surfaces import build_hstrat_surfaces
from pylib._build_path import build_path
from pylib._make_seed_coordinates import make_seed_coordinates
from pylib._pathsplot import pathsplot
from pylib.auxlib._hstrat_reconstruct_dataframe import (
    hstrat_reconstruct_dataframe,
)
from pylib.auxlib._log_context_duration import log_context_duration
from pylib.auxlib._make_hstrat_surface_dataframe import (
    make_hstrat_surface_dataframe,
)
from pylib.auxlib._make_reproducible import make_reproducible
from pylib.auxlib._shift_no_wrap import shift_no_wrap
from pylib.auxlib._transform_box_max import TransformBoxMax
from pylib.synthgeom._AttributeBoxMax import AttributeBoxMax
from pylib.synthgeom._VideoWriter import VideoWriter
from pylib.synthgeom._generate_observations import generate_observations

make_reproducible()


In [ ]:
%watermark -diwmuv -iv


In [ ]:
SX, SY = 400, 400
crea_func = np.ones
crea_size = 10
num_frames = 1000
dstream_algo = "dstream.tilted_algo"
dstream_S = 64


def mirror_backtrack_transform(arr, i):
    return shift_no_wrap(arr, (min(i, SX // 4), -min(i, SY // 4)), axis=(0, 1))


In [ ]:
def show_sample_observations(
    SX,
    SY,
    *,
    crea_func,
    crea_size=crea_size,
    every_nth=5,
    fps=30.0,
    logger=print,
    mirror_backtrack=False,
    mirror_backtrack_transform=mirror_backtrack_transform,
    num_frames=1000,
):

    observations = generate_observations(
        SX=SX,
        SY=SY,
        crea_func=crea_func,
        crea_size=crea_size,
        mirror_backtrack=mirror_backtrack,
        mirror_backtrack_transform=mirror_backtrack_transform,
        num_frames=num_frames,
    )
    with log_context_duration("Creating video", logger=logger):
        with VideoWriter("out.gif", fps) as vid:
            vid.add_observations(observations, every_nth=every_nth)
            vid.show()

    with log_context_duration("Creating reversed video", logger=logger):
        with VideoWriter("out.gif", fps) as vid:
            vid.add_observations(
                observations, every_nth=every_nth, reorder=reversed
            )
            vid.show()

    return observations


In [ ]:
def show_sample_hstrat_tree(
    obvs,
    SX,
    SY,
    *,
    dstream_algo,
    dstream_S,
    num_frames,
):
    surfaces = build_hstrat_surfaces(
        obvs,
        (SY, SX),
        TransformBoxMax(10),
        dstream_algo=dstream_algo,
        dstream_S=dstream_S,
    )
    surfaces_df = make_hstrat_surface_dataframe(
        surfaces[-1],
        num_frames + dstream_S,
        values=obvs[-1],
        dstream_algo=dstream_algo,
    )
    surfaces_phylo = hstrat_reconstruct_dataframe(
        surfaces_df[surfaces_df["value"] > 0.2],
    )
    surfaces_phylo["ancestor_list"] = [
        "[]" if id_ == ancestor_id else f"[{ancestor_id}]"
        for (id_, ancestor_id) in zip(
            surfaces_phylo["id"],
            surfaces_phylo["ancestor_id"],
        )
    ]
    draw_colorclade_tree(
        surfaces_phylo,
        taxon_name_key="id",
        backend="biopython",
    )
    plt.show()


In [ ]:
seed_coordinates = make_seed_coordinates(
    x_coords=range(SX // 8, SX, SX // 4),
    y_coords=range(SY // 8, SY, SY // 4),
)


## `crea_func=np.ones` `mirror_backtrack=False`


In [ ]:
obvs = show_sample_observations(
    SX,
    SY,
    crea_func=np.ones,
    mirror_backtrack=False,
    num_frames=num_frames,
)
observations = pathsplot(
    [
        build_path(coordinate, obvs, AttributeBoxMax(20))
        for coordinate in tqdm(seed_coordinates)
    ],
    xmax=SX,
    ymax=SY,
    col_wrap=4,
)
plt.show()

show_sample_hstrat_tree(
    obvs,
    SX,
    SY,
    dstream_algo=dstream_algo,
    dstream_S=dstream_S,
    num_frames=num_frames,
)


## `crea_func=np.ones` `mirror_backtrack=True`


In [ ]:
obvs = show_sample_observations(
    SX,
    SY,
    crea_func=np.ones,
    mirror_backtrack=True,
    mirror_backtrack_transform=mirror_backtrack_transform,
    num_frames=num_frames,
)
observations = pathsplot(
    [
        build_path(coordinate, obvs, AttributeBoxMax(20))
        for coordinate in tqdm(seed_coordinates)
    ],
    xmax=SX,
    ymax=SY,
    col_wrap=4,
)
plt.show()

show_sample_hstrat_tree(
    obvs,
    SX,
    SY,
    dstream_algo=dstream_algo,
    dstream_S=dstream_S,
    num_frames=num_frames,
)


## `crea_func=np.random.rand` `mirror_backtrack=False`


In [ ]:
for i in range(2):
    obvs = show_sample_observations(
        SX,
        SY,
        crea_func=lambda x: np.random.rand(*x),
        mirror_backtrack=False,
        num_frames=num_frames,
    )
    observations = pathsplot(
        [
            build_path(coordinate, obvs, AttributeBoxMax(20))
            for coordinate in tqdm(seed_coordinates)
        ],
        xmax=SX,
        ymax=SY,
        col_wrap=4,
    )
    plt.show()

show_sample_hstrat_tree(
    obvs,
    SX,
    SY,
    dstream_algo=dstream_algo,
    dstream_S=dstream_S,
    num_frames=num_frames,
)


## `crea_func=np.random.rand` `mirror_backtrack=True`


In [ ]:
for i in range(2):
    obvs = show_sample_observations(
        SX,
        SY,
        crea_func=lambda x: np.random.rand(*x),
        mirror_backtrack=True,
        mirror_backtrack_transform=mirror_backtrack_transform,
        num_frames=num_frames,
    )
    observations = pathsplot(
        [
            build_path(coordinate, obvs, AttributeBoxMax(20))
            for coordinate in tqdm(seed_coordinates)
        ],
        xmax=SX,
        ymax=SY,
        col_wrap=4,
    )
    plt.show()

show_sample_hstrat_tree(
    obvs,
    SX,
    SY,
    dstream_algo=dstream_algo,
    dstream_S=dstream_S,
    num_frames=num_frames,
)
